In [ ]:
!git clone https://github.com/ZYChimne/AI-Debater.git
!pip install transformers datasets

fatal: destination path 'AI-Debater' already exists and is not an empty directory.


In [ ]:
from datasets import load_dataset
from datasets import Dataset
import pandas as pd
import csv
import torch
from torch import nn
from transformers import AutoTokenizer, Trainer, TrainingArguments
from transformers.models.bert.modeling_bert import BertPreTrainedModel, BertModel
from sklearn.metrics import accuracy_score
from transformers.modeling_outputs import SequenceClassifierOutput
model_name = 'hfl/chinese-roberta-wwm-ext'
batch_size = 8

In [ ]:
class MyBertForSequenceClassification(BertPreTrainedModel):
    _keys_to_ignore_on_load_missing = [r"position_ids"]

    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.config = config

        self.bert = BertModel(config, add_pooling_layer=False)
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)
        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
        use_y = True
    ):
        r"""
        labels (:obj:`torch.LongTensor` of shape :obj:`(batch_size,)`, `optional`):
            Labels for computing the sequence classification/regression loss. Indices should be in :obj:`[0, ...,
            config.num_labels - 1]`. If :obj:`config.num_labels == 1` a regression loss is computed (Mean-Square loss),
            If :obj:`config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        x = outputs[0]
        x = x[:, 0, :]  # take <s> token (equiv. to [CLS])
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        if use_y:
          logits_x = self.dropout(x)
          logits_y = self.dropout(x)
          # logits_z = self.dropout(x)
          logits_x = self.out_proj(logits_x)
          logits_y = self.out_proj(logits_y)
          # logits_z = self.out_proj(logits_z)
          loss = None
          logits = (logits_x + logits_y)/2
          if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            # loss_x = loss_fct(logits_x.view(-1, self.num_labels), labels.view(-1))
            # loss_y = loss_fct(logits_y.view(-1, self.num_labels), labels.view(-1))
            # loss_xy_fct = nn.MSELoss()
            # loss_xy_fct = nn.KLDivLoss()
            # loss_xy = loss_xy_fct(logits_x, logits_y)
            # loss = (loss_x + loss_y)/2
            # loss = loss + loss_xy
        else:
          logits = self.dropout(x)
          logits = self.out_proj(logits)
          loss = None
          if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
  
        if not return_dict:
          output = (logits,) + outputs[2:]
          return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutput(
          loss=loss,
          logits=logits,
          hidden_states=outputs.hidden_states,
          attentions=outputs.attentions,
        )

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = MyBertForSequenceClassification.from_pretrained(model_name, num_labels=3)

Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext were not used when initializing MyBertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'bert.pooler.dense.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'bert.pooler.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing MyBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MyBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MyBertFor

In [ ]:
path = 'AI-Debater/track1_phase1/data/'
train_df = pd.read_csv(path+'train.txt', sep='\t', header=None, quoting=csv.QUOTE_NONE)
test_df = pd.read_csv(path+'test.txt', sep='\t', header=None, quoting=csv.QUOTE_NONE)
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
def process_dataset(dataset):
  dataset = dataset.map(lambda e: tokenizer(e['0'], e['1'], truncation=True, padding='max_length', max_length=256), batched=True)
  def map_label(e):
    d = {'Support':0, 'Neutral':1, 'Against':2}
    e['label']=d[e['2']]
    return e
  dataset = dataset.map(map_label)
  dataset = dataset.remove_columns(['0', '1', '2'])
  dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
  print(dataset)
  return dataset
train_dataset = process_dataset(train_dataset)
test_dataset = process_dataset(test_dataset)


Dataset({
    features: ['attention_mask', 'input_ids', 'label', 'token_type_ids'],
    num_rows: 6416
})



Dataset({
    features: ['attention_mask', 'input_ids', 'label', 'token_type_ids'],
    num_rows: 990
})


In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc}

In [ ]:
training_args = TrainingArguments(
    overwrite_output_dir=True,
    output_dir='./results',          
    num_train_epochs=5,              
    logging_dir='./logs', 
    per_device_train_batch_size=batch_size,
    fp16=True,
    # evaluation_strategy='epoch',
    evaluation_strategy='steps',
    eval_steps = 100,
    logging_steps = 100, 
    learning_rate=5e-5
)
trainer = Trainer(
    model=model,                        
    args=training_args,                 
    train_dataset=train_dataset,        
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
) 

In [ ]:
!nvidia-smi
trainer.train()

Wed Jun 23 09:35:21 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    32W /  70W |   1510MiB / 15109MiB |     12%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Step,Training Loss,Validation Loss,Accuracy
100,0.784400,0.673519,0.740404
200,0.460400,0.936772,0.727273
300,0.537000,0.663466,0.739394
400,0.475500,0.573726,0.753535
500,0.423600,1.101159,0.705051
600,0.499900,0.807671,0.731313
700,0.452300,1.075724,0.686869
800,0.395400,0.733455,0.748485
900,0.275700,1.062675,0.737374
1000,0.304100,1.134460,0.725253


/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1293: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  args.max_grad_norm,
/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1293: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  args.max_grad_norm,
/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1293: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior wil

TrainOutput(global_step=4010, training_loss=0.19555772469005087, metrics={'train_runtime': 1525.9739, 'train_samples_per_second': 21.023, 'train_steps_per_second': 2.628, 'total_flos': 5039339760230400.0, 'train_loss': 0.19555772469005087, 'epoch': 5.0})